In [ ]:
import pandas as pd
import akshare as ak
import seaborn as sns
import arch
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from arch import arch_model
warnings.filterwarnings('ignore')
sns.set(font='SimHei')
plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 
df = ak.stock_zh_a_spot_em()
df.to_excel('example.xlsx', sheet_name='Sheet1', index=False)

In [ ]:
import akshare as ak

stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol="688484", period="daily", start_date="20170301", end_date='20231022', adjust="")
print(stock_zh_a_hist_df)

In [ ]:
import pandas as pd
import akshare as ak
df=ak.stock_zh_a_hist(symbol='688484', period='daily', start_date='20230601', end_date='20240601', adjust='qfq')
df.to_csv('688484abc.csv',index=False)

In [ ]:
df['date']=pd.to_datetime(df['日期'])
df=df.set_index('date')

In [ ]:
ret=100*df['收盘'].pct_change().dropna()

In [ ]:
from statsmodels.tsa.stattools import adfuller #ADF单位根检验
result = adfuller(ret) #不能拒绝原假设，即原序列存在单位根
print(result)

划分训练集和预测部分

In [ ]:
ret_train=ret[ret.index<'2024-05-01']
ret_test=ret[ret.index>='2024-05-01']

建立简单arch模型，进行arch效应检验

In [ ]:
t1=arch.arch_model(ret_train, mean="Constant", vol="ARCH", p=1)
result=t1.fit(disp="off")

In [ ]:
result

In [ ]:
result.arch_lm_test()

In [ ]:
from statsmodels.stats.diagnostic import het_arch
from scipy.stats import shapiro

In [ ]:
# 模型优化
def gridsearch(data, p_rng, q_rng):
    top_score, top_results = float('inf'), None
    top_models = []
    for p in range(len(p_rng)):
        for q in range(len(q_rng)):
            model = arch_model(data, vol='GARCH', p=p_rng[p], q=q_rng[q], dist='normal')
            model_fit = model.fit(disp='off')
            resid = model_fit.resid
            st_resid = np.divide(resid, model_fit.conditional_volatility)
            results = evaluate_model(resid, st_resid)
            results['AIC'] = model_fit.aic
            results['params']['p'] = p+1
            results['params']['q'] = q+1
            print(model_fit.aic)
            if results['AIC'] < top_score:
                top_score = results['AIC']
                top_results = results

    top_models.append(top_results)
    return top_models


# 模型评估
def evaluate_model(residuals, st_residuals, lags=50):
    results = {
        'LM_pvalue': None,
        'F_pvalue': None,
        'SW_pvalue': None,
        'AIC': None,
        'params': {'p': None, 'q': None}
    }

    arch_test = het_arch(residuals, nlags=lags)
    shap_test = shapiro(st_residuals)

    results['LM_pvalue'] = [arch_test[1], arch_test[1] < 0.05]
    results['F_pvalue'] = [arch_test[3], arch_test[3] < 0.05]
    results['SW_pvalue'] = [shap_test[1], shap_test[1] < 0.05]

    return results


p_rng = list(range(1, 5)) 
q_rng = list(range(1, 5)) 
top_models = gridsearch(ret_train, p_rng, q_rng)
print('*****************top_models*******************')
print(top_models)

In [ ]:
garch_model_1=arch.arch_model(ret_train,vol='GARCH',p=3,q=3)
garch_result=garch_model_1.fit(disp="off")

In [ ]:
garch_result.hedgehog_plot()

In [ ]:
forecasts_train = garch_result.forecast(start=0,horizon=1)
forecasts_train.residual_variance

In [ ]:
forecast_test=garch_result.forecast(horizon=len(ret_test))

In [ ]:
tmp=forecast_test.residual_variance.dropna().T
tmp2=tmp[tmp.columns[0]].values.tolist()
future=pd.DataFrame(tmp2)
future.columns=['h.1']
future.index=ret_test.index

In [ ]:
forecasts_train.residual_variance

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(forecasts_train.residual_variance.index,forecasts_train.residual_variance.values,color='gray')
plt.plot(future.index,future[future.columns[0]],color='red')
ret.plot(color = "blue", zorder = 3)
plt.title("Predictions of GARCH Model", size=24)
plt.legend(['train predict','test predict','actual'])
plt.show()

In [ ]:
am = arch_model(ret, vol="arch", p=3, o=0, q=3, dist="skewt")
res = am.fit(disp="off")

In [ ]:
forecasts = res.forecast(start=0)
cond_mean = forecasts.mean.dropna()
cond_var = forecasts.variance.dropna()
q = am.distribution.ppf([0.01, 0.05], res.params[-2:])
print(q)

In [ ]:
value_at_risk = -cond_mean.values - np.sqrt(cond_var).values * q[None, :]
value_at_risk = pd.DataFrame(value_at_risk, columns=["1%", "5%"], index=cond_var.index)

In [ ]:
for col in value_at_risk.columns:
    value_at_risk[col]=-value_at_risk[col]

In [ ]:
ax = value_at_risk.plot(legend=False)
xl = ax.set_xlim(value_at_risk.index[0], value_at_risk.index[-1])
rets_2018 = ret.copy()
rets_2018.name = "S&P 500 Return"


c = []
for idx in value_at_risk.index:
    if rets_2018[idx] > value_at_risk.loc[idx, "5%"]:
        c.append("#000000")
    elif rets_2018[idx] < value_at_risk.loc[idx, "1%"]:
        c.append("#BB0000")
    else:
        c.append("#BB00BB")
c = np.array(c, dtype="object")
labels = {
    "#BB0000": "1% Exceedence",
    "#BB00BB": "5% Exceedence",
    "#000000": "No Exceedence",
}
markers = {"#BB0000": "x", "#BB00BB": "s", "#000000": "o"}
for color in np.unique(c):
    sel = c == color
    ax.scatter(
        rets_2018.index[sel],
        rets_2018.loc[sel],
        marker=markers[color],
        c=c[sel],
        label=labels[color],
    )
ax.set_title("Parametric VaR")
leg = ax.legend(frameon=False, ncol=3)

In [ ]:
forecasts_train.residual_variance

In [ ]:
t=pd.DataFrame(future)
t.columns=['h.1']

In [ ]:
vol=forecasts_train.residual_variance._append(t)

In [ ]:
vol['ret']=ret

In [ ]:
vol.plot()

In [ ]:
vol_tmp=vol[vol.index>='2024-05-01']

In [ ]:
total=[]
first_price=df['收盘'].values.tolist()[-25]
next_price=first_price
for val in vol_tmp['h.1'].values.tolist():
    next_price=(1+(val/100))*next_price
    total.append(next_price)      

In [ ]:
vol_tmp['h.1'].values.tolist()

In [ ]:
df_tmp=df[df.index>='2024-05-01']
df_tmp['pred_garch']=total
df

In [ ]:
df_tmp['pred_garch'].plot()
df_tmp['收盘'].plot()
plt.legend(['pred close price','actual close price'])